## SONAR: Emotionally Intelligent Music Recommendations

Draws heavily from [this tutorial](https://ahmedbesbes.com/sentiment-analysis-on-twitter-using-word2vec-and-keras.html)

Welcome to Sonar! This project was inpired me to first explore Machine Learning. The premise is simple: we often want different music to listen to based on our mood. However, oftentimes finding that music proves difficult. Sonar takes away that difficulty: by entering short mood messages, sonar generates Spotify recommendations using both your old favorites and determined mood. 

Enjoy!

In [1]:
import re

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool
import dill as pkl
import gensim
from gensim.models.word2vec import Word2Vec
from keras import Sequential
from keras.layers import Dense, Activation
import matplotlib.pyplot as plt
import numpy as np
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm_notebook, tqdm

#tqdm_notebook().pandas(desc="progress-bar")
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
TaggedDocument = gensim.models.doc2vec.TaggedDocument
STOPWORDS = set(stopwords.words('english'))

not_stops = ["but", "very", "most", "nor", "no", "against"]
for stop in not_stops:
    STOPWORDS.remove(stop)
    
%matplotlib inline

Using TensorFlow backend.


### Preprocessing

- Load tweets into a usable format. Clean up null values and reindex.

- Tokenize the data and take only the n values we aim to use in training.

In [2]:
def load_tweets():
    tweets_train = pd.read_csv("/home/malits/data/tweets/training.1600000.processed.noemoticon.csv", 
                               encoding = "ISO-8859-1")
    data = pd.DataFrame()
    data["sentiment"] = tweets_train["0"].apply(lambda v: 1 if (v >= 2) else 0)
    data["sentiment"] = data["sentiment"].map(int)
    data["tweet"] = tweets_train.iloc[:, [5]]
    data["tweet"] = data["tweet"].astype(str)
    data = data[data['tweet'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    
    return data

In [3]:
data = load_tweets()
data.head()

,sentiment,tweet
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [4]:
def postprocess(data, n=1000000):
    data = data.head(n)
    data.loc[:, "tokens"] = data.loc[:, "tweet"].apply(tokenize)
    data = data.loc[data.tokens != 'NC', :]
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

In [5]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

def expand_contractions(text):
    try:
        return contractions[text]
    except KeyError:
        return text

In [6]:
def tokenize(tweet):
    sub_regex = r'(#[A-Za-z0-9])|(https)|[\?!\.,;:]'
    tweet = str(tweet)
    tokens = tokenizer.tokenize(tweet)
    tokens = [re.sub(sub_regex, '', t) for t in tokens]
    tokens = [t.lower() for t in tokens if t is not None and t != '']
    tokens = [expand_contractions(t) for t in tokens]
    tokens = [t for t in tokens if t not in STOPWORDS]
    return tokens

In [7]:
processed = postprocess(data, len(data))

/home/malits/anaconda3/envs/sonar/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/malits/anaconda3/envs/sonar/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
processed.head(5)

,sentiment,tweet,tokens
0,0,is upset that he can't update his Facebook by ...,"[upset, cannot, update, facebook, texting, mig..."
1,0,@Kenichan I dived many times for the ball. Man...,"[dived, many, times, ball, managed, save, 50, ..."
2,0,my whole body feels itchy and like its on fire,"[whole, body, feels, itchy, like, fire]"
3,0,"@nationwideclass no, it's not behaving at all....","[no, it has / it is, behaving, i'm, mad, canno..."
4,0,@Kwesidei not the whole crew,"[whole, crew]"


### Create training data

Once preprocessed, split to test/train sets and vectorize w w2vec for use in keras

In [9]:
# Size of our embedding space

N_DIM = 200

In [10]:
X_train, X_test, y_train, y_test = train_test_split(np.array(processed['tokens']),
                                                    np.array(processed['sentiment']),
                                                    test_size=0.2)

In [11]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i, v in tqdm_notebook(enumerate(tweets)):
        label = '{}_{}'.format(label_type, i)
        ## Use nltk TaggedDocument obj to mark words with their corresponding label. Makes words easier to retrieve from their vector form.
        labelized.append(TaggedDocument(v, [label]))
    return labelized

In [12]:
X_train = labelizeTweets(X_train, 'TRAIN')
X_test = labelizeTweets(X_test, 'TEST')

## Replace Word2Vec corpus with more data!

In [13]:
train_size = len(X_train)
tweet_w2v = Word2Vec(size=N_DIM, min_count=10)
tweet_w2v.build_vocab([x.words for x in tqdm_notebook(X_train)])
tweet_w2v.train([x.words for x in tqdm_notebook(X_train)], 
                total_examples=train_size, epochs = tweet_w2v.epochs)

(44582499, 50683860)

In [14]:
def get_similarity(text):
    return tweet_w2v.wv.most_similar(text.lower())

In [15]:
get_similarity('kanye')

[('all-american', 0.6761536598205566),
 ('ne-yo', 0.6727895140647888),
 ('indies', 0.6499416828155518),
 ('leighton', 0.6405508518218994),
 ('rejects', 0.6387357115745544),
 ('feat', 0.6371992230415344),
 ('barry', 0.6328898668289185),
 ('sings', 0.6260355710983276),
 ('adele', 0.6210954189300537),
 ('covina', 0.6206552386283875)]

In [16]:
keys = list(tweet_w2v.wv.vocab.keys())[:5000]

word_vectors = [tweet_w2v[w] for w in keys]

tsne_model = TSNE(n_components = 2, verbose = 1, random_state = 10)
tsne_w2v = tsne_model.fit_transform(word_vectors)

tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = keys

/home/malits/anaconda3/envs/sonar/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 5000 samples in 0.026s...
[t-SNE] Computed neighbors for 5000 samples in 8.004s...
[t-SNE] Computed conditional probabilities for sample 1000 / 5000
[t-SNE] Computed conditional probabilities for sample 2000 / 5000
[t-SNE] Computed conditional probabilities for sample 3000 / 5000
[t-SNE] Computed conditional probabilities for sample 4000 / 5000
[t-SNE] Computed conditional probabilities for sample 5000 / 5000
[t-SNE] Mean sigma: 0.547245
[t-SNE] KL divergence after 250 iterations with early exaggeration: 85.230515
[t-SNE] KL divergence after 1000 iterations: 2.371732


In [17]:
tsne_df.head(4)

,x,y,words
0,-1.923984,-26.506626,im
1,36.973930,-26.296928,trying
2,-5.716387,44.449276,get
3,41.921158,27.877148,mine


In [18]:
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,"

p = figure(tools=TOOLS)

p.scatter(x='x', y='y', source=tsne_df)

hover = p.select(dict(type=HoverTool))
hover.tooltips = [
    ("word", "@words"),
]

output_notebook()

show(p)

Loading BokehJS ...

## Transforming

We use TF-IDF weightings to weight word vectors accordingly

In [19]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [20]:
def create_tweet_vector(words, ndims=200):
    vector = np.zeros((1, ndims))
    count = 0
    
    for w in words:
        try:
            vector += tweet_w2v.wv[w].reshape((1, ndims)) * tfidf[w]
            count += 1
        except KeyError:
            continue
    
    if count != 0:
        return np.divide(vector,count)
    else:
        return vector

In [21]:
train_vec_tweets = np.concatenate([create_tweet_vector(x, N_DIM) for x in tqdm_notebook(map(lambda x: x.words, X_train))])
train_vec_tweets = scale(train_vec_tweets)

test_vec_tweets = np.concatenate([create_tweet_vector(x, N_DIM) for x in tqdm_notebook(map(lambda x: x.words, X_test))])
test_vec_tweets = scale(test_vec_tweets)

## Neural Architecture in Keras

In [23]:
model = Sequential([
    Dense(32, input_dim=N_DIM),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid')
])
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.fit(train_vec_tweets, y_train, epochs=9, batch_size=32, verbose=2)

score = model.evaluate(test_vec_tweets, y_test, batch_size=32)

Epoch 1/9
 - 59s - loss: 0.4766 - acc: 0.7730
Epoch 2/9
 - 64s - loss: 0.4666 - acc: 0.7792
Epoch 3/9
 - 80s - loss: 0.4640 - acc: 0.7805
Epoch 4/9
 - 74s - loss: 0.4620 - acc: 0.7817
Epoch 5/9
 - 79s - loss: 0.4609 - acc: 0.7823
Epoch 6/9
 - 80s - loss: 0.4600 - acc: 0.7826
Epoch 7/9
 - 80s - loss: 0.4594 - acc: 0.7831
Epoch 8/9
 - 77s - loss: 0.4589 - acc: 0.7832
Epoch 9/9
 - 66s - loss: 0.4585 - acc: 0.7835
320000/320000 [==============================] - 7s 23us/step


In [24]:
score

[0.4600269847497344, 0.78199375]

In [25]:
def predict_text(texts):
    texts = [tokenize(text) for text in texts]
    matrix = np.concatenate([create_tweet_vector(t, N_DIM) for t in texts])
    return model.predict(matrix)

In [26]:
predict_text(["I hate everything. Today sucks!", "I love myself. Life is awesome!"])

array([[0.00129254],
       [0.9999833 ]], dtype=float32)

## Pickling the neural model, w2vec model and tfidf lookup

In [31]:
pkl.dump(model, open("../data/sentiment_classifier00.pkl", "wb"))
pkl.dump(tweet_w2v, open("../data/tweet_w2v00.pkl", "wb"))
pkl.dump(tfidf, open("../data/tfidf_lookup00.pkl", "wb"))
pkl.dump(tokenizer, open("../data/tweet_tokenizer00.pkl", "wb"))

## Productionalizing Code

In [37]:
class SentimentAnalyzer:
    
    def __init__(self):
        self.ndims = 200
    
    def load_model(self):
        self.classifier = pkl.load(open("../data/sentiment_classifier00.pkl", "rb"))
        self.word2vec = pkl.load(open("../data/tweet_w2v00.pkl", "rb"))
        self.tfidf = pkl.load(open("../data/tfidf_lookup00.pkl", "rb"))
        self.tokenizer = pkl.load(open("../data/tweet_tokenizer00.pkl", "rb"))
        
    def tokenize(self, tweet):
        sub_regex = r'(#[A-Za-z0-9])|(https)\?!\.,'
        tweet = str(tweet)
        tokens = self.tokenizer.tokenize(tweet)
        tokens = [re.sub(sub_regex, '', t) for t in tokens]
        tokens = [t.lower() for t in tokens if t is not None and t != '']
        tokens = [self.expand_contractions(t) for t in tokens]
        tokens = [t for t in tokens if t not in STOPWORDS]
        return tokens
        
    def expand_contractions(self, text):
        try:
            return contractions[text]
        except:
            return text
        
    def create_tweet_vector(self, words):
        vector = np.zeros((1, self.ndims))
        count = 0

        for w in words:
            try:
                vector += self.word2vec.wv[w].reshape((1, self.ndims)) * self.tfidf[w]
                count += 1
            except KeyError:
                continue

        if count != 0:
            return np.divide(vector,count)
        else:
            return vector
    
    def predict_sentiment(self, texts):
        texts = [self.tokenize(text) for text in texts]
        tweet_vectors = np.concatenate([self.create_tweet_vector(text) for text in texts])
        return self.classifier.predict(tweet_vectors)


In [38]:
sa = SentimentAnalyzer()
sa.load_model()

In [39]:
sa.predict_sentiment(["NOT GOOD"])

array([[0.9999856]], dtype=float32)